In [1]:
# Summary: the code is FEniCS-based implementation of Stokes and contiuum equations 
#    with mechanical anisotropy described in Maulhaus et al. (2002). 
# Part of the code is adopted from the FEniCS Stokes-iterative demo, whose link is provided below:
#    https://fenicsproject.org/olddocs/dolfin/1.3.0/python/demo/documented/stokes-iterative/python/documentation.html

# Now version 1.1.9. 
# Created on 08/02/2021. Last modfied on 06/29/2021. 
# Author: Dunyu Liu (dliu@ig.utexas.edu) 

In [2]:
from dolfin import *
import numpy as np
import ufl as ufl
import sys
import math
import matplotlib.pyplot as plt
from lib import *
from prepare_case import *

%matplotlib inline
#sys.setrecursionlimit(1500)

# Test for PETSc or Epetra
if not has_linear_algebra_backend("PETSc") and not has_linear_algebra_backend("Epetra"):
    info("DOLFIN has not been configured with Trilinos or PETSc. Exiting.")
    exit()

if not has_krylov_solver_preconditioner("amg"):
    info("Sorry, this code is only available when DOLFIN is compiled with AMG "
	 "preconditioner, Hypre or ML.");
    exit()

In [3]:
case = 32
# case 30 for 3D box with a horziontal anisotropic layer.
# case 31 for 3D box with vertical SAF fault zone parallel to boundaries.
# case 32 for 
date_stamp = '20220629' # Date stamp is used in the output folder name.

In [4]:
def stokes(delta, eta_strong, eta_weak, msh_path = None, msh_name = None, n = 10):
    
    # Use function prepare_case to generate key model specific data.
    mesh, boundaries, mf, bcs, f, bound_name_list, dim, W = prepare_case(case, msh_path, msh_name, n)
    #------- Case specific modifications end here --------
    
    anis_domain = bound_name_list['anis_domain']
    iso_domain = bound_name_list['iso_domain']
    
    ds = Measure("ds")(domain=mesh, subdomain_data = boundaries)
    
    if eta_strong>eta_weak:
        hete = 1
        iso = False
    elif eta_strong == eta_weak: 
        hete = 0
        iso = True

    # Define eta and eta_s.
    # eta is the strong viscosity in the anisotropic cases or the viscosity of isotropic case.
    # eta_s is the weak viscosity in the anisotropic cases.
    if iso == True:
        eta = K(mf, eta_weak, eta_strong, anis_domain, iso_domain, degree=0)
    else:
        eta = eta_strong
    eta_s = K(mf, eta_weak, eta_strong, anis_domain, iso_domain, degree=0) 
    
    # Define functions -----------------------------------
    def epsilon(v):
        return sym(nabla_grad(v))
    # Define stress
    def sigma(v,p,eta):
        return 2*eta*sym(nabla_grad(v))-(-p)*Identity(dim)
    
    # https://www.continuummechanics.org/principalstressesandstrains.html
    # calc_princ is derived from the above link.
    def calc_princ(sigma):
        thetap = atan(2*sigma[0,1]/(sigma[0,0] - sigma[1,1]))
        thetap = thetap/math.pi*180/2

        sigma1 = (sigma[0,0] + sigma[1,1])/2 + sqrt(pow((sigma[0,0]/2 - sigma[1,1]/2),2) + sigma[0,1]*sigma[0,1])
        sigma3 = (sigma[0,0] + sigma[1,1])/2 - sqrt(pow((sigma[0,0]/2 - sigma[1,1]/2),2) + sigma[0,1]*sigma[0,1])

        return thetap, sigma1, sigma3
    
    # Define anisotropic viscosity constitutive relation
    # Based on Maulhaus et al. (2002). 
    def sigma_anisotropic(v, eta, eta_s, dim):
        if dim == 2:
            n = np.zeros((1,dim))
            n[0][0] = norm1
            n[0][1] = norm3       
        elif dim == 3:
            n = np.zeros((1,dim))
            n[0][0] = norm1
            n[0][1] = norm2
            n[0][2] = norm3
        C_lambda = np.zeros((dim, dim, dim, dim))

        componentList = []
        for i in range(dim):
            componentList += [[],]
            for j in range(dim):
                componentList[i] += [[],]
                for k in range(dim):
                    componentList[i][j] += [[],]
                    for l in range(dim):
                        componentList[i][j][k] += [[],]    

        for i in range(dim):
            for j in range(dim):
                for k in range(dim):
                    for l in range(dim):
                        a = 0
                        b = 0
                        c = 0
                        d = 0
                        e = n[0][i]*n[0][j]*n[0][k]*n[0][l]
                        if l == j :
                            a = n[0][i]*n[0][k]
                        if i == l :
                            b = n[0][j]*n[0][k]
                        if k == j :
                            c = n[0][i]*n[0][l]
                        if i == k :
                            d = n[0][j]*n[0][l]
                        componentList[i][j][k][l] = 2*(eta_s - eta) * ((a + b + c + d)/2 - 2*e)
                        #C_lambda[i][j][k][l] = (a + b + c + d)/2 - 2*e

        C = as_tensor(componentList)
        vtmp = as_tensor(epsilon(v))
        i, j, k, l = ufl.indices(4)
        C1 = ufl.as_tensor(C[i,j,k,l]*vtmp[k,l],(i,j))
        C2 = 2*eta*sym(nabla_grad(v))
        C3 = C1 + C2
        return C3

    # Define variational problem
    (u, p) = TrialFunctions(W)
    (v, q) = TestFunctions(W)
    
    theta = 90 + delta
    norm1 = cos(theta/180*math.pi)
    norm2 = sin(theta/180*math.pi)
    norm3 = 0    

    print('Normal vectors for anisotrpy are ', norm1, norm2, norm3, 'in this scenario ...')
    
    if iso:
        theta2 = -10000
        nametag = '_isotropic' # Appended in the result files.
        a = inner(grad(u), eta*grad(v))*dx + div(v)*p*dx + q*div(u)*dx
        L = inner(f, v)*dx
        # b is used to build the preconditioner matrix
        b = inner(grad(u), eta*grad(v))*dx + p*q*dx
    else:
        nametag = '_anisotropic'
        a1 = inner(sigma_anisotropic(u,eta,eta_s,dim), grad(v))*dx
        a2 = div(v)*p*dx + q*div(u)*dx
        a = a1 + a2
        L = inner(f, v)*dx
        # b is used to build the preconditioner matrix
        b = inner(grad(u), 2*eta*grad(v))*dx + p*q*dx    

    # Assemble the system
    A, bb = assemble_system(a, L, bcs)
    # Assemble the preconditioner system
    P, btmp = assemble_system(b, L, bcs)
    print('Assembling the system and preconditioner ...')

    # Create Krylov solver and AMG preconditioner
    solver = KrylovSolver("minres", "amg") # best.
    #solver = KrylovSolver("gmres", "amg") # second best.
    #solver = KrylovSolver("tfqmr", "amg") # least

    solver.parameters["relative_tolerance"] = 1.0e-9
    solver.parameters["absolute_tolerance"] = 1.0e-15
    solver.parameters["divergence_limit"] = 1.0e4
    solver.parameters["maximum_iterations"] = 7000
    solver.parameters["error_on_nonconvergence"] = True
    solver.parameters["nonzero_initial_guess"] = False
    solver.parameters["report"] = True
    solver.parameters["monitor_convergence"] = True
    # Associate operator (A) and preconditioner matrix (P)
    solver.set_operators(A, P)

    # Solve
    U = Function(W)
    
    print('Solving the system, be patient...')

    solver.solve(U.vector(), bb)
    
    # Get sub-functions
    u, p = U.split()

    # Calculate stress and strain-rate tensors
    Vsig = TensorFunctionSpace(mesh, "DG", degree=0) # Set DG0 for stress & strain rate
    sig = Function(Vsig, name="Stress")
    strain_rate = Function(Vsig, name="Strain")

    # Project stresses to Vsig space.
    if iso==True:
        sig.assign(project(sigma(u,p,eta), Vsig))
    else: 
        sig.assign(project(sigma_anisotropic(u,eta,eta_s,dim)-(-p)*Identity(dim), Vsig))

    strain_rate.assign(project(epsilon(u), Vsig))

    print('Writing out the results ...')
    # Save solution in xdmf format
    ufile_pvd = XDMFFile("../res/case"+ str(case) + "/" + str(date_stamp) + "/velocity_theta"+"{:.1f}".format(delta) + "_hetero_"+ str(hete) + ".xdmf")
    ufile_pvd.write(u)
    pfile_pvd = XDMFFile("../res/case"+ str(case) + "/" + str(date_stamp) + "/pressure_theta"+"{:.1f}".format(delta) + "_hetero_"+ str(hete) + ".xdmf")
    pfile_pvd.write(p)
    pfile_pvd = XDMFFile("../res/case"+ str(case) + "/" + str(date_stamp) + "/stress_theta"+"{:.1f}".format(delta) + "_hetero_"+ str(hete) + ".xdmf")
    pfile_pvd.write(sig)
    pfile_pvd = XDMFFile("../res/case" + str(case) + "/" + str(date_stamp) + "/strain_rate_theta"+"{:.1f}".format(delta) + "_hetero_"+ str(hete) + ".xdmf")
    pfile_pvd.write(strain_rate)
    
    print('Finished ...')

In [5]:
# theta is the angle of the layer normal to the horizontal axis in the square model.
# theta should be >90 and <180.
# delta = theta - 90.

if case == 30: 
    # Figure 4b & 6.
    n = 10
    eta_strong, eta_weak = 1, 0.1
    msh_path = None
    msh_name = None
    delta = 10
    alpha = 45
    stokes(delta, eta_strong, eta_weak, msh_path, msh_name, n)
        
if case == 31:
    # Figure 4a & 5.
    n = 10
    eta_strong, eta_weak = 1, 0.1
    msh_path = None
    msh_name = None
    delta = 12.5
    stokes(delta, eta_strong, eta_weak, msh_path, msh_name, n)
        
if case == 32:
    # Figure 7 & 8.
    msh_path = "../msh/Cascadia_schist_mesh_case32/"
    msh_name = "cascadia_thrust"
    eta_strong, eta_weak = 1, 0.1
    delta = 30
    stokes(delta, eta_strong, eta_weak, msh_path, msh_name, n=10)

Simulating Case 32: Simplified 3D box model with the Leech River Schist above Cascadia Subduction Zone
Normal vectors for anisotrpy are  -0.4999999999999998 0.8660254037844387 0 in this scenario ...
Assembling the system and preconditioner ...
Solving the system, be patient...
Solving linear system of size 135655 x 135655 (PETSc Krylov solver).
  0 KSP preconditioned resid norm 6.812575048822e+01 true resid norm 1.499271243179e+02 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP preconditioned resid norm 8.066867405237e+00 true resid norm 8.743482981780e+00 ||r(i)||/||b|| 5.831821974547e-02
  2 KSP preconditioned resid norm 5.429174752283e+00 true resid norm 3.449761433581e+00 ||r(i)||/||b|| 2.300958848691e-02
  3 KSP preconditioned resid norm 5.341755992274e+00 true resid norm 3.648720921447e+00 ||r(i)||/||b|| 2.433662979962e-02
  4 KSP preconditioned resid norm 1.846038843825e+00 true resid norm 1.713200360130e+00 ||r(i)||/||b|| 1.142688734893e-02
  5 KSP preconditioned resid norm 1.6555714

Writing out the results ...
Finished ...
